In [6]:
import pandas as pd
df = pd.read_csv(r'H:\GEN_AI _Bappy\Data_processing\IMDB Dataset.csv')

In [7]:
df = df[:100]

# Cleaning up Data

In [8]:
import re
import string
import emoji
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer

# Precompile patterns and load resources once
HTML_PATTERN = re.compile('<.*?>')
URL_PATTERN = re.compile(r'https?://\S+|www\.\S+')
STOPWORDS = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def get_pos(tag):
    """Map POS tag to WordNet POS for lemmatization."""
    if tag.startswith('J'): return wordnet.ADJ
    elif tag.startswith('V'): return wordnet.VERB
    elif tag.startswith('N'): return wordnet.NOUN
    elif tag.startswith('R'): return wordnet.ADV
    else: return wordnet.NOUN

def preprocess_text(text):
    # Lowercase
    text = text.lower()
    # Remove HTML/URLs
    text = HTML_PATTERN.sub('', text)
    text = URL_PATTERN.sub('', text)
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenize and remove stopwords
    words = word_tokenize(text)
    words = [word for word in words if word not in STOPWORDS]
    # Lemmatize with POS tagging
    pos_tags = pos_tag(words)
    words = [lemmatizer.lemmatize(word, pos=get_pos(tag)) for (word, tag) in pos_tags]
    # Handle emojis (remove or keep)
    #text = emoji.replace_emoji(" ".join(words), replace='')
    #text = emoji.replace_emoji(" ".join(words), replace='emoji')
    text = emoji.demojize(" ".join(words))
    return text

df['review'] = df['review'].apply(preprocess_text)

# Word2vector

In [39]:
!pip install --upgrade gensim --user

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.



  Using cached numpy-1.26.4-cp310-cp310-win_amd64.whl.metadata (61 kB)
   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
   -- ------------------------------------- 1.6/24.0 MB 9.4 MB/s eta 0:00:03
   ------ --------------------------------- 4.2/24.0 MB 10.5 MB/s eta 0:00:02
   -------- ------------------------------- 5.2/24.0 MB 7.2 MB/s eta 0:00:03
   ----------- ---------------------------- 7.1/24.0 MB 7.3 MB/s eta 0:00:03
   ------------- -------------------------- 7.9/24.0 MB 6.8 MB/s eta 0:00:03
   ------------- -------------------------- 8.4/24.0 MB 6.1 MB/s eta 0:00:03
   -------------- ------------------------- 8.9/24.0 MB 5.5 MB/s eta 0:00:03
   --------------- ------------------------ 9.4/24.0 MB 5.2 MB/s eta 0:00:03
   ---------------- ----------------------- 10.0/24.0 MB 5.0 MB/s eta 0:00:03
   ----------------- ---------------------- 10.5/24.0 MB 4.8 MB/s eta 0:00:03
   ------------------ --------------------- 11.3/24.0 MB 4.5 MB/s eta 0:00:03
   ----

In [1]:
import gensim

In [4]:
import numpy as np
import pandas as pd
import os
from nltk.tokenize import sent_tokenize

In [47]:
a = df['review'].tolist()

In [60]:
story = []
for i in range(len(a)):
    raw_sent = word_tokenize(a[i])
    story.append(raw_sent)

In [81]:
story

[['one',
  'reviewer',
  'mention',
  'watch',
  '1',
  'oz',
  'episode',
  'youll',
  'hook',
  'right',
  'exactly',
  'happen',
  'methe',
  'first',
  'thing',
  'strike',
  'oz',
  'brutality',
  'unflinching',
  'scene',
  'violence',
  'set',
  'right',
  'word',
  'go',
  'trust',
  'show',
  'faint',
  'hearted',
  'timid',
  'show',
  'pull',
  'punch',
  'regard',
  'drug',
  'sex',
  'violence',
  'hardcore',
  'classic',
  'use',
  'wordit',
  'call',
  'oz',
  'nickname',
  'give',
  'oswald',
  'maximum',
  'security',
  'state',
  'penitentary',
  'focus',
  'mainly',
  'emerald',
  'city',
  'experimental',
  'section',
  'prison',
  'cell',
  'glass',
  'front',
  'face',
  'inwards',
  'privacy',
  'high',
  'agenda',
  'em',
  'city',
  'home',
  'manyaryans',
  'muslims',
  'gangsta',
  'latino',
  'christian',
  'italian',
  'irish',
  'moreso',
  'scuffle',
  'death',
  'stare',
  'dodgy',
  'dealing',
  'shady',
  'agreement',
  'never',
  'far',
  'awayi',
  '

In [62]:
model = gensim.models.Word2Vec(window=10,min_count=2)
model.build_vocab(story)

In [63]:

model.train(story, total_examples=model.corpus_count, epochs=model.epochs)

(42811, 60320)

In [86]:
vocab = model.wv.key_to_index.keys()
len(vocab)

1660

In [84]:
a = model.wv.get_normed_vectors()

In [85]:
a.shape

(1660, 100)